In [1]:
import csv, sqlite3

con = sqlite3.connect("my_db.db")
cur = con.cursor()

In [2]:
%load_ext sql

In [4]:
%sql sqlite:///my_db.db

In [3]:
import pandas as pd

# ***FORMATTING!***

In [5]:
#changing CSV files into SQL tables

prods = pd.read_csv('olist_products_dataset.csv')
prods.to_sql("products", con, if_exists='replace', index=False)

sellers = pd.read_csv('olist_sellers_dataset.csv')
sellers.to_sql("sellers", con, if_exists='replace', index=False)

orders = pd.read_csv('olist_orders_dataset.csv')
orders.to_sql("orders", con, if_exists='replace', index=False)

ordit = pd.read_csv('olist_order_items_dataset.csv')
ordit.to_sql("o_t", con, if_exists='replace', index=False)

cust = pd.read_csv('olist_customers_dataset.csv')
cust.to_sql("cust", con, if_exists='replace', index=False)

trans = pd.read_csv('product_category_name_translation.csv')
trans.to_sql("trans", con, if_exists='replace', index=False)


71

In [7]:
##Creating organized tables

%%sql
CREATE TABLE order_det AS
SELECT
    SUBSTR(order_id,-5,5) AS o_id,
    SUBSTR(customer_id,-5,5) AS c_id,
    order_status AS o_st,
    STRFTIME('%d/%m/%Y',SUBSTR(order_purchase_timestamp, 1, 10)) AS date,
    STRFTIME('%d/%m/%Y',SUBSTR(order_delivered_customer_date, 1, 10)) AS del_date
FROM orders ;

CREATE TABLE prod_det AS
SELECT
    SUBSTR(product_id,-5,5) AS prod_id,
    product_category_name AS prod_name,
    product_weight_g AS weight,
    product_length_cm AS length,
    product_height_cm AS height,
    product_width_cm AS width

FROM products;

CREATE TABLE cust_det AS
SELECT
    SUBSTR(customer_unique_id,-5,5) AS cu_id,
    SUBSTR(customer_id,-5,5) AS c_id,
    customer_city AS city,
    customer_state AS state,
    customer_zip_code_prefix AS zipcode
FROM cust;

CREATE TABLE sellers_det AS
SELECT
    SUBSTR(seller_id,-5,5) AS prod_id,
    seller_zip_code_prefix AS seller_zip,
    seller_city AS seller_city,
    seller_state AS seller_state

FROM sellers;

CREATE TABLE ordit_det AS
SELECT
    SUBSTR(order_id,-5,5) AS order_id,
    SUBSTR(product_id,-5,5) AS prod_id,
    SUBSTR(seller_id,-5,5) AS seller_id,
    STRFTIME('%d/%m/%Y',SUBSTR(shipping_limit_date, 1, 10)) AS ship_date,
    price AS price,
    freight_value AS freight

FROM o_t;




 * sqlite:///my_db.db
Done.
Done.
Done.
Done.
Done.


[]

In [8]:
##changing products' spanish names with english names

%%sql
UPDATE prod_det
SET prod_name = (SELECT product_category_name_english FROM trans WHERE trans.product_category_name = prod_det.prod_name)
WHERE EXISTS (SELECT 1 FROM trans WHERE trans.product_category_name = prod_det.prod_name);

 * sqlite:///my_db.db
32328 rows affected.


[]

## ***Reports!***

In [38]:
# Sales per state report
%%sql
CREATE TABLE sales_per_state AS
SELECT s.seller_city AS city, COUNT(o.prod_id) AS 'Sales Count Per City'
FROM ordit_det o
LEFT JOIN sellers_det s ON o.prod_id = s.prod_id
where s.seller_city IS NOT NULL
GROUP BY s.seller_city;

 * sqlite:///my_db.db
Done.


[]

In [30]:
# Most sold items per state
%%sql

create table most_sold_per_state as
SELECT seller_city AS City, prod_name AS Product, MAX(sales_count) AS Count
FROM (
    SELECT p.prod_name, s.seller_city, COUNT(o.prod_id) AS sales_count
    FROM ordit_det o
    LEFT JOIN sellers_det s ON o.prod_id = s.prod_id
    LEFT JOIN prod_det p ON o.prod_id = p.prod_id
    WHERE s.seller_city IS NOT NULL
    AND p.prod_name IS NOT NULL
    GROUP BY p.prod_name, s.seller_city
) AS subquery
GROUP BY seller_city

 * sqlite:///my_db.db
Done.


[]

In [39]:
# Converting sql files to excel files
# List of tables to export
tables = ['ordit_det', 'sellers_det', 'cust_det', 'prod_det', 'order_det','sales_per_state', 'most_sold_per_state']

# Export each table to a CSV file
for table in tables:
    # Read data from the table into a pandas DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {table}", con)

    # Write the DataFrame to a CSV file
    df.to_csv(f"{table}.csv", index=False)


In [40]:
ordit_det = pd.read_csv('ordit_det.csv')
order_det = pd.read_csv('order_det.csv')
prod_det = pd.read_csv('prod_det.csv')
cust_det = pd.read_csv('cust_det.csv')
sellers_det = pd.read_csv('sellers_det.csv')
sales_per_state = pd.read_csv('sales_per_state.csv')
most_sold_per_state = pd.read_csv('most_sold_per_state.csv')

In [ ]:
#checking the number of null values in each table

print("order_det null count\n",order_det.isnull().sum())
print("ordit_det count\n",ordit_det.isnull().sum())
print("prod_det null count\n",prod_det.isnull().sum())
print("cust_det null count\n",cust_det.isnull().sum())
print("sellers_det null count\n",sellers_det.isnull().sum())
print("sales_per_state null count\n",sales_per_state.isnull().sum())
print("most_sold_per_state null count\n",most_sold_per_state.isnull().sum())

In [43]:
sales_per_state.head(10)

,city,Sales Count Per City
0,alvares machado,1
1,americana,6
2,arapongas,1
3,araras,1
4,belo horizonte,7
5,campinas,2
6,campo largo,2
7,campos novos,27
8,canoinhas,6
9,cascavel,1
